In [ ]:
# Common imports
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
data = pd.read_csv("Google_Stock_Price_Train.csv")
data_test = pd.read_csv("Google_Stock_Price_Test.csv")
data_Open = data.loc[:,["Open"]].values
data_test_Open = data_test.loc[:,["Open"]].values
train = data_Open[:len(data)-200] 
valid = data_Open[len(train):] 
train = train.reshape(train.shape[0],1)
valid = valid.reshape(valid.shape[0],1)
test = data_test_Open.reshape(data_test_Open.shape[0],1)
scaler = MinMaxScaler(feature_range= (0,1)) # defining of Scaler
train_scaled = scaler.fit_transform(train) # applying to Scaler to train
test_scaled = scaler.transform(test)
valid_scaled = scaler.transform(valid)

In [ ]:
X_valid = []
y_valid = []
timesteps = 120

for i in range(timesteps, valid_scaled.shape[0]):
    X_valid.append(valid_scaled[i-timesteps:i,0])
    y_valid.append(valid_scaled[i,0])

X_valid, y_valid = np.array(X_valid), np.array(y_valid)


# Reshaping
X_valid = X_valid.reshape(X_valid.shape[0], X_valid.shape[1], 1)  # Dimension of array is 3.


In [ ]:
X_train = []
y_train = []
#timesteps = 50

for i in range(timesteps, train_scaled.shape[0]):
    X_train.append(train_scaled[i-timesteps:i,0])
    y_train.append(train_scaled[i,0])

X_train, y_train = np.array(X_train), np.array(y_train)


# Reshaping
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # Dimension of array is 3.

In [ ]:
dataset_total = pd.concat((data['Open'], data_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(data_test) - timesteps:].values
inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)
X_test = []
for i in range(timesteps, inputs.shape[0]):
    X_test.append(inputs[i-timesteps:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
sweep_config = {

     'method': 'grid',

    'parameters': {
        'units': 
         {'values': [20,30,40,50,60,80,120,150]},
        
        'Drop_1': 
            {'values': [ 0.1,0.2,0.25,0.3]
                   
                   
                },
        
        'Drop_2': 
            {'values': [ 0.1,0.2,0.3,0.4,0.5]
                   
                   
                }
     
    
        }

    }
sweep_id = wandb.sweep(sweep_config, project= "LSTM_4Layers") 

In [ ]:
def train():
    
    
    
    
    from tensorflow.keras.models import Sequential  
    from tensorflow.keras.layers import Dense 
    from tensorflow.keras.layers import SimpleRNN
    from tensorflow.keras.layers import LSTM
    from tensorflow.keras.layers import Dropout # it block to overfitting 
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.callbacks import ModelCheckpoint

    wandb.init()
    configs = {'units': 80,'Drop_1': 0.25, 'Drop_2': 0.5}
    config = wandb.config
    config.epochs = 2000
    config.Batchsize = 256
    
    
    
    model = Sequential()
    model.add(LSTM(units = wandb.config.units,return_sequences = True,input_shape = (X_train.shape[1],1)))
    model.add(Dropout(wandb.config.Drop_1))

    model.add(LSTM(units = wandb.config.units,return_sequences = True))
    model.add(Dropout(wandb.config.Drop_1))

    model.add(LSTM(units = wandb.config.units,return_sequences = True))
    model.add(Dropout(wandb.config.Drop_1))


    model.add(LSTM(units = wandb.config.units))
    model.add(Dropout(wandb.config.Drop_2))

    model.add(Dense(1))
    
    es = EarlyStopping(monitor= 'val_loss', mode='min', patience=200)

#     metric = "val_accuracy"

    modelsave = ModelCheckpoint("LSTM_best.h5", monitor= 'val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='min')
    
    
    #n_epochs = 100
    batch = 256
    #opt = Adam(lr=wandb.config.learning_rate,amsgrad=True)
    
    #opt = Adam(learning_rate=3e-3 , beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
    model.compile(loss='mean_squared_error', optimizer="Adam")
    
    wandb_callback = WandbCallback(validation_data=(X_valid, y_valid))

    History_LSTM = model.fit(X_train, y_train, epochs=wandb.config.epochs,validation_data=(X_valid, y_valid)
                         , batch_size=wandb.config.Batchsize,shuffle=False
                             , callbacks=[es,modelsave,wandb_callback])


In [ ]:
wandb.agent(sweep_id, function=train)